In [2]:
import pandas as pd
from utils import connection as con
from etl import extract as ex
from etl import load
from etl import transform as t

In [4]:
conn_db = con.connect_db()
conn_dw = con.connect_dw()

In [5]:
# Pasamos los datos de las tablas a su DataFrame

df_categoria = ex.extract_only_table("Categoria", conn_db)
df_subcategoria = ex.extract_only_table("SubCategoria", conn_db)
df_carta = ex.extract_only_table("Carta", conn_db)

df_mesa = ex.extract_only_table("Mesa", conn_db)
df_clientes = ex.extract_only_table("Clientes", conn_db)
df_distrito = ex.extract_only_table("Distrito", conn_db).rename(columns={'Descripcion' : 'Distrito'})
df_direccion = ex.extract_only_table("Direcciones", conn_db)
df_local = ex.extract_only_table("Local", conn_db)
df_trabajador = ex.extract_only_table("Trabajador", conn_db)

df_pedidosdetalle = ex.extract_only_table("PedidosDetalle", conn_db)


Reemplazando IDs por descripciones

In [6]:
# Reemplazamos el idDistrito por su descripción.
df_direccion_dw = df_direccion.merge(df_distrito, on='idDistrito', how='left')[['idDirecciones','Descripcion','Distrito','idClientes','Estado']]

df_local_dw = df_local.merge(df_distrito, on='idDistrito', how='left')[['idLocal','NombreLocal','Direccion','Distrito','HorarioAtencion']]


In [7]:
# Reemplazamos el idCategoria y idSubCategoria para la carta y ponemos sus descripciones.
df_carta_dw = df_carta.merge(df_subcategoria.rename(columns={'nombre' : 'SubCategoria'}), on='idSubCategoria', how='left')\
                      .merge(df_categoria.rename(columns={'Nombre' : 'Categoria'}), on='idCategoria', how='left')\
                      [['idCarta','Nombre','Descripcion','Precio','Estado','Categoria','SubCategoria']]


In [8]:
# Reemplazamos el idLocal por su descripción.
df_trabajador_dw = df_trabajador.merge(df_local.rename(columns={'NombreLocal' : 'Local'}), on='idLocal', how='left')\
                                [['idTrabajador','ApellidosNombres','Local']]


Cargando los datos a las dimensiones

In [98]:
load.load_dw("Dim_Carta", conn_dw, df_carta_dw)

135

In [9]:
load.load_dw("Dim_Mesa", conn_dw, df_mesa)

In [10]:
load.load_dw("Dim_Local", conn_dw, df_local_dw)

In [11]:
load.load_dw("Dim_Trabajador", conn_dw, df_trabajador_dw)

In [12]:
load.load_dw("Dim_Clientes", conn_dw, df_clientes)

In [13]:
load.load_dw("Dim_Direcciones", conn_dw, df_direccion_dw)

In [12]:
query = """
SELECT DISTINCT 
DATE_FORMAT(Fecha, '%%Y%%m%%d') AS idTiempo,
YEAR(Fecha) AS Anio,
MONTH(Fecha) AS Mes,
DAY(Fecha) AS Dia,
QUARTER(Fecha) AS Trimestre,
DAYOFWEEK(Fecha) AS DiaSemana,
DAYOFYEAR(Fecha) AS DiaAnio,
WEEK(Fecha) AS SemanaAnio,
CASE WHEN DAYOFWEEK(Fecha) IN (1,7) THEN TRUE ELSE FALSE END AS FinSemana
FROM Pedidos
"""

df_dim_tiempo = t.transform_time_from_pedido(query, conn_db)

In [13]:
load.load_dw("Dim_Tiempo", conn_dw, df_dim_tiempo)

In [14]:
load.load_dw("Dim_PedidosDetalle", conn_dw, df_pedidosdetalle)

(MySQLdb.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`dw_restaurante`.`Dim_PedidosDetalle`, CONSTRAINT `fk_PedidosDetalle_Pedidos1` FOREIGN KEY (`idPedidos`) REFERENCES `Hec_Fact_Pedidos` (`idPedidos`))')
[SQL: INSERT INTO `Dim_PedidosDetalle` (`idPedidos`, `idCarta`, `Cantidad`, `PrecioUnitario`, `Precio`) VALUES (%s, %s, %s, %s, %s)]
[parameters: [(1, 4, 1, 29.0, 29.0), (1, 5, 1, 29.0, 29.0), (1, 39, 1, 58.0, 58.0), (1, 42, 1, 98.0, 98.0), (1, 96, 2, 34.0, 68.0), (2, 22, 1, 17.0, 17.0), (2, 38, 1, 69.0, 69.0), (2, 39, 1, 58.0, 58.0)  ... displaying 10 of 26 total bound parameter sets ...  (5, 76, 1, 22.0, 22.0), (5, 117, 2, 12.0, 24.0)]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [8]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
  print(x)

apple
banana
cherry


In [ ]:
df_join  = orders_df.merge(order_items_df, left_on='order_id', right_on='order_item_order_id')\
                    .merge(products_df, left_on='order_item_product_id', right_on='product_id')\
                    .merge(categories_df, left_on='product_category_id', right_on='category_id')\
                    .merge(departments_df, left_on='category_department_id', right_on='department_id')
df_join